In [52]:
import json 
import openai 

client = openai.Client()

In [53]:
def obter_temperatura_atual(local:str, unidade="celsius"):
    cidades = {
		"São paulo": 32,
		"Porto Alegre": 25,
		"Minas Gerais": 33
	}
    
    return json.dumps({"local": local, "temperatura": cidades.get(local)})
    
obter_temperatura_atual("Minas Gerais")

'{"local": "Minas Gerais", "temperatura": 33}'

In [54]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "obter_temperatura_atual",
            "description": "Obtém a temperatura atual em uma dada cidade",
            "parameters": {
                "type": "object",
                "properties": {
                    "local": {
                        "type": "string",
                        "description": "O nome da cidade. Ex: São Paulo",
                    },
                    "unidade": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["local"],
            },
        },
    }
]

In [55]:
mensagens = [{"role": "user", "content": "Qual a temperatura de Minas Gerais?"}]

res = client.chat.completions.create(
	model='gpt-3.5-turbo',
	messages=mensagens,
	tools=tools,
	
)

res

ChatCompletion(id='chatcmpl-9u2lCSBm8pDo3dUPCdBHL2Z0zlluR', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_bM2oqkvipySzd6OAXHCdymYm', function=Function(arguments='{"local":"Minas Gerais","unidade":"celsius"}', name='obter_temperatura_atual'), type='function')], refusal=None))], created=1723144006, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=27, prompt_tokens=86, total_tokens=113))

In [56]:
mensagem = res.choices[0].message

In [57]:
mensagem.content

In [58]:
mensagem.tool_calls

[ChatCompletionMessageToolCall(id='call_bM2oqkvipySzd6OAXHCdymYm', function=Function(arguments='{"local":"Minas Gerais","unidade":"celsius"}', name='obter_temperatura_atual'), type='function')]

In [59]:
tools_call = mensagem.tool_calls[0]
print(tools_call.function.name)
print(tools_call.function.arguments)

obter_temperatura_atual
{"local":"Minas Gerais","unidade":"celsius"}


In [60]:
observacao = obter_temperatura_atual(**json.loads(tools_call.function.arguments))
observacao

'{"local": "Minas Gerais", "temperatura": 33}'

In [61]:
mensagens.append(mensagem)

In [62]:
mensagens.append({
    'tool_call_id': tools_call.id,
    'role': 'tool',
    'name': tools_call.function.name,
    'content': observacao
})
mensagens

[{'role': 'user', 'content': 'Qual a temperatura de Minas Gerais?'},
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_bM2oqkvipySzd6OAXHCdymYm', function=Function(arguments='{"local":"Minas Gerais","unidade":"celsius"}', name='obter_temperatura_atual'), type='function')], refusal=None),
 {'tool_call_id': 'call_bM2oqkvipySzd6OAXHCdymYm',
  'role': 'tool',
  'name': 'obter_temperatura_atual',
  'content': '{"local": "Minas Gerais", "temperatura": 33}'}]

In [63]:
mensagens

[{'role': 'user', 'content': 'Qual a temperatura de Minas Gerais?'},
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_bM2oqkvipySzd6OAXHCdymYm', function=Function(arguments='{"local":"Minas Gerais","unidade":"celsius"}', name='obter_temperatura_atual'), type='function')], refusal=None),
 {'tool_call_id': 'call_bM2oqkvipySzd6OAXHCdymYm',
  'role': 'tool',
  'name': 'obter_temperatura_atual',
  'content': '{"local": "Minas Gerais", "temperatura": 33}'}]

In [64]:
resposta = client.chat.completions.create(
    model='gpt-3.5-turbo-0125',
    messages=mensagens,
    tools=tools,
    tool_choice='auto'
)
resposta

ChatCompletion(id='chatcmpl-9u2lDRRfuetRvstPjwdADt7FZueY5', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='A temperatura atual em Minas Gerais é de 33°C.', role='assistant', function_call=None, tool_calls=None, refusal=None))], created=1723144007, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=15, prompt_tokens=142, total_tokens=157))

In [65]:
resposta.choices[0].message.content

'A temperatura atual em Minas Gerais é de 33°C.'